In [12]:
#Import all the necessary packages
import numpy as np
import tensorflow as tf
import edward as ed
from edward.models import Normal, BernoulliWithSigmoidProbs, Bernoulli, Dirichlet, Empirical
from edward.util import Progbar
import time
import math
import matplotlib.pyplot as pyplot
from sklearn.metrics import roc_curve, auc
import random

In [2]:
def split_train_test(DATA, y, proportion=0.8):
    data_size = len(DATA)
    train_size = int(proportion * data_size)
    permutation = [i for i in range(data_size)]
    random.shuffle(permutation)
    train_DATA = DATA[permutation[:train_size]]
    train_label = y[permutation[:train_size]]
    test_DATA = DATA[permutation[train_size:]]
    test_label = y[permutation[train_size:]]
    return train_DATA, train_label, test_DATA, test_label

In [3]:
#Generating a data set according to the PSD graphical model
#N: the number of SNPs considered
#M: the numer of individuals
#K: the number of populations/topics, as each individual is a mixture of them
#return the data set
def generate_PSD_without_label(N, M, K):
    alpha = np.ones(K) / K
    eta = np.ones(2) / 2
    theta = np.random.dirichlet(alpha, size=M)
    beta = np.random.dirichlet(eta, size=(K, N))
    DATA = []
    gamma = np.random.normal(size=(K,))
    labels = []
    y_raws = []
    for m in range(M):
        Z_tmp = np.random.multinomial(1, theta[m], size=(2, N))
        Z = np.zeros(shape=(2, N), dtype='int32')
        X = np.zeros(N, dtype='int32')
        y_raw = 0
        for _ in range(2):
            for n in range(N):
                Z[_][n] = np.argmax(Z_tmp[_][n])
                p = beta[Z[_][n]][n]
                tmp = 1 if np.random.random() <= p[0] else 0
                X[n] += tmp
        DATA.append(X)
    y_raws = theta.dot(gamma)
    y_raws = y_raws / np.std(y_raws)
    probas = 1 / (1 + np.exp(-y_raws))
    labels = (np.random.random(M) < probas).astype(int).reshape(M, 1)
    DATA = np.array(DATA, dtype='float')
    Z = np.array(Z)
    return DATA, Z, theta, beta

In [4]:
#Generating a data set according to the PSD graphical model
#N: the number of SNPs considered
#M: the numer of individuals
#K: the number of populations/topics, as each individual is a mixture of them
#return the data set and labels dependent on the mixtures and SNPs
def generate_PSD_with_label(N, M, K):
    alpha = np.ones(K) / K
    eta = np.ones(2) / 2
    theta = np.random.dirichlet(alpha, size=M)
    beta = np.random.dirichlet(eta, size=(K, N))
    DATA = []
    gamma = np.random.normal(size=(K,N))
    labels = []
    y_raws = []
    for m in range(M):
        Z_tmp = np.random.multinomial(1, theta[m], size=(2, N))
        Z = np.zeros(shape=(2, N), dtype='int32')
        X = np.zeros(N, dtype='int32')
        y_raw = 0
        for _ in range(2):
            for n in range(N):
                Z[_][n] = np.argmax(Z_tmp[_][n])
                p = beta[Z[_][n]][n]
                tmp = 1 if np.random.random() <= p[0] else 0
                X[n] += tmp
                y_raw += tmp * gamma[Z[_][n]][n]
        DATA.append(X)
        y_raws.append(y_raw)
    y_raws = y_raws / np.std(y_raws)
    probas = 1 / (1 + np.exp(-y_raws))
    labels = (np.random.random(M) < probas).astype(int).reshape(M, 1)
    DATA = np.array(DATA, dtype='float')
    Z = np.array(Z)
    gold_roc_curve = roc_curve(labels, probas)
    gold_auc = auc(gold_roc_curve[0], gold_roc_curve[1])
    return DATA, labels, Z, theta, beta, gold_roc_curve, gold_auc

In [5]:
#Whenever we inference on populations, since it is unsupervised learning,
#the order of the inferred population is always a problem
#Matching the inferred population
def get_best_match(theta_truth, theta_inferred):
    assert (theta_truth.shape == theta_inferred.shape)
    M = theta_truth.shape[0]
    K = theta_truth.shape[1]
    #A brute force search, cannot handle cases for K >=6
    assert (K <= 6)
    permutations = generate_permutation([i for i in range(K)])
    best_permutation = None
    min_rmse = 1
    for permutation in permutations:
        theta_permuted = theta_truth[:,permutation]
        rmse = np.sqrt(((np.sum(np.square(theta_permuted - theta_inferred))) / M / K))
        if rmse < min_rmse:
            best_permutation = permutation
            min_rmse = rmse
    return best_permutation, min_rmse
        
    
#Return a list of list, each list is a permutation
def generate_permutation(l):
    if len(l) == 1:
        return [[l[0]]]
    result = []
    for element in l:
        copy = l[:]
        copy.remove(element)
        suffix_list = generate_permutation(copy)
        for suffix in suffix_list:
            result.append([element] + suffix)
    return result

# Approximating PSD Decomposition using Bayesian Auto-encoder

In [21]:
def approximate_PSD_with_bayesian_AE(DATA, latent_dim, coef_var, inference_method='KLqp'):
    sample_size = DATA.shape[0]
    SNP_count = DATA.shape[1]
    w = Normal(loc=tf.zeros([SNP_count, latent_dim]), 
               scale=coef_var * tf.ones([SNP_count, latent_dim]))
    z = Dirichlet(tf.ones([sample_size, latent_dim]) / latent_dim)
    x = tf.add(BernoulliWithSigmoidProbs(tf.matmul(w, z, transpose_b=True)), 
                   BernoulliWithSigmoidProbs(tf.matmul(w, z, transpose_b=True)))
    if inference_method=='KLqp':
        qw = Normal(loc=tf.Variable(tf.random_normal([SNP_count, latent_dim])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([SNP_count, latent_dim]))))
        #The random initialization coefficient 0.01 is hand-picked here
        #In general, KLqp in this setting is extremely fragile; a slightly larger
        #coefficient will result in nan in loss
        #Even 0.01 sometimes does not work; the edward seed is also manually picked
        #to make a working example
        qz = Dirichlet(tf.Variable((tf.zeros([sample_size, latent_dim]) 
                               + tf.random_uniform([sample_size, latent_dim]) * 0.01)
                               / latent_dim))
        inference = ed.KLqp({w: qw, z: qz}, data={x: DATA.T})
        #We have also tried different optimizers; however, none of them is better than
        #the built-in
        #optimizer = tf.train.RMSPropOptimizer(0.01)
        #optimizer = tf.train.AdamOptimizer()
        #inference.initialize(optimizer=optimizer)
        inference.run(n_iter=50000, n_print=100, n_samples=100)
        return qw, qz
    else:
        T = 5000
        qw = Empirical(params=tf.Variable(tf.zeros([T, SNP_count, latent_dim])))
        qz = Empirical(params=tf.Variable(tf.zeros([T, sample_size, latent_dim])))
        inference = None
        if inference_method == 'HMC':
            inference = ed.HMC({w: qw, z: qz}, {x: DATA.T})
        elif inference_method == 'Gibbs':
            inference = ed.Gibbs({w: qw, z: qz}, {x: DATA.T})
        inference.run(step_size=0.5 / sample_size, n_steps=10)
        return qw, qz
    

In [7]:
np.random.seed(0)
random.seed(0)
ed.set_seed(1)
N, M, K = 100, 10, 3
alpha = np.ones(K) / K
DATA, Z, theta, beta = generate_PSD_without_label(N, M, K)

In [8]:
tf.reset_default_graph()
coef_var = 2
qw, qz = approximate_PSD_with_bayesian_AE(DATA, K, coef_var)

50000/50000 [100%] ██████████████████████████████ Elapsed: 157s | Loss: 650.100


In [9]:
inferred_theta_mean = np.mean(qz.sample(10000).eval(), axis=0)
print(inferred_theta_mean)

[[ 0.30678025  0.34347826  0.34974   ]
 [ 0.28955489  0.4168213   0.29362366]
 [ 0.36547545  0.36289462  0.27163002]
 [ 0.34359479  0.3145403   0.34186471]
 [ 0.3109794   0.36745816  0.32156312]
 [ 0.29541248  0.37457874  0.33000848]
 [ 0.37628451  0.31937417  0.30434155]
 [ 0.28919294  0.30996752  0.40083873]
 [ 0.35698453  0.3219856   0.32102877]
 [ 0.37249371  0.27115145  0.35635456]]


In [10]:
best_permutation, min_rmse = get_best_match(theta, inferred_theta_mean)
baseline_theta = np.ones([M, K]) / K
baseline = np.sqrt(np.sum(np.square(baseline_theta - theta)) / M / K)
print('Baseline rmse %f' % baseline)
print('Result obtained by KLqp: %f' % min_rmse)

Baseline rmse 0.419915
Result obtained by KLqp: 0.410329


In [24]:
#There is a reported bug of HMC cannot get the value of latent dirichlet dimension correct
#After I have implemented the model, I found that other people have also complained about
#it before
qw, qz = approximate_PSD_with_bayesian_AE(DATA, K, coef_var, inference_method='HMC')

ValueError: Dimensions must be equal, but are 3 and 4 for 'inference_3/sample_102/Dirichlet_9/log_prob/mul' (op: 'Mul') with input shapes: [10,3], [10,4].

In conclusion, this model does not work in Edward, since there is no robust and effective inference algorithm for it. Furthermore, it is still unknown whether it will be a reasonable approximition even if we are able to calculate the posterior.

# Unsupervised Probablisitc PCA followed by Bayesian Logistics Regression

In [9]:
#Unsupervised Probablistic Dimension Reduction
def unsupervised_dimension_reduction(DATA, latent_dim, dimension_reduction_coef_var, 
                                    observation_var, sigmoid=False):
    sample_size = DATA.shape[0]
    SNP_count = DATA.shape[1]
    if sigmoid:
        observation_var = None
    w = Normal(loc=tf.zeros([SNP_count, latent_dim]), 
               scale=dimension_reduction_coef_var * tf.ones([SNP_count, latent_dim]))
    z = Normal(loc=tf.zeros([sample_size, latent_dim]), 
               scale=tf.ones([sample_size, latent_dim]))
    if not sigmoid:
        x = Normal(loc=tf.matmul(w, z, transpose_b=True), 
                   scale=observation_var * tf.ones([SNP_count, sample_size]))
    else:
        x = tf.add(BernoulliWithSigmoidProbs(tf.matmul(w, z, transpose_b=True)), 
                   BernoulliWithSigmoidProbs(tf.matmul(w, z, transpose_b=True)))
    qw = Normal(loc=tf.Variable(tf.random_normal([SNP_count, latent_dim])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([SNP_count, latent_dim]))))
    qz = Normal(loc=tf.Variable(tf.random_normal([sample_size, latent_dim])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([sample_size, latent_dim]))))
    inference = ed.KLqp({w: qw, z: qz}, data={x: DATA.T})
    inference.run(n_iter=500, n_print=100, n_samples=100)
    return qw, qz

In [10]:
def learn_latent_given_w(DATA, qw, latent_dim, observation_var, sigmoid=False):
    sample_size = DATA.shape[0]
    SNP_count = DATA.shape[1]
    z = Normal(loc=tf.zeros([sample_size, latent_dim]), 
               scale=tf.ones([sample_size, latent_dim]))
    if not sigmoid:
        x = Normal(loc=tf.matmul(qw, z, transpose_b=True), 
                   scale=observation_var * tf.ones([SNP_count, sample_size]))
    else:
        x = tf.add(BernoulliWithSigmoidProbs(tf.matmul(qw, z, transpose_b=True)), 
                   BernoulliWithSigmoidProbs(tf.matmul(qw, z, transpose_b=True)))
    qz = Normal(loc=tf.Variable(tf.random_normal([sample_size, latent_dim])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([sample_size, latent_dim]))))
    inference = ed.KLqp({z: qz}, data={x: DATA.T})
    inference.run(n_iter=500, n_print=100, n_samples=10)
    return qz

In [11]:
def bayesian_logistics_regression(qz, y_labels, latent_dim, regression_coef_var):
    regression_coef = Normal(loc=tf.zeros([1, latent_dim]), 
               scale=regression_coef_var * tf.ones([1, latent_dim]))
    y = Bernoulli(logits=tf.matmul(qz, regression_coef, transpose_b=True))
    qcoeff = Normal(loc=tf.Variable(tf.random_normal([1, latent_dim])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([1, latent_dim]))))
    inference = ed.KLqp({regression_coef: qcoeff}, data={y: y_labels})
    inference.run(n_iter=500, n_print=100, n_samples=10)
    return qcoeff

In [12]:
def dimension_reduction_and_logistics_regression(latent_dim, dimension_reduction_coef_var, 
                                                 regression_coef_var, sigmoid, 
                                                 observation_var, DATA, labels):
    if sigmoid:
        observation_var = None
    train_DATA, train_label, test_DATA, test_label = split_train_test(DATA, labels)
    test_label = test_label.reshape((test_label.shape[0],))
    qw, qz_train = unsupervised_dimension_reduction(train_DATA, latent_dim, 
                                              dimension_reduction_coef_var, 
                                              observation_var, sigmoid)
    qcoef = bayesian_logistics_regression(qz_train, train_label, latent_dim, regression_coef_var)
    qz_test = learn_latent_given_w(test_DATA, qw, latent_dim, observation_var, sigmoid)
    n_samples=100
    probas = tf.gather(tf.reduce_mean(tf.stack([tf.sigmoid(tf.matmul(qz_test.sample(), qcoef.sample(), transpose_b=True))
                  for _ in range(n_samples)]), axis=0), 0, axis=1)
    score = probas.eval()
    fpr, tpr, thresholds = roc_curve(test_label, score)
    roc_area = auc(fpr, tpr)
    
    return roc_area
    

In [13]:
N, M, K = 1000, 100, 3
DATA, labels, Z, theta, beta, gold_roc_curve, gold_auc = generate_PSD_with_label(N, M, K)
DATA[0][0] = np.nan
DATA[1][0] = np.nan
DATA[3][0] = np.nan
DATA[5][2] = np.nan

In [14]:
roc_area = dimension_reduction_and_logistics_regression(50, 1, 1, True, 1, DATA, labels)
print(roc_area)

500/500 [100%] ██████████████████████████████ Elapsed: 9s | Loss: 0.000
500/500 [100%] ██████████████████████████████ Elapsed: 5s | Loss: 7.011
500/500 [100%] ██████████████████████████████ Elapsed: 2s | Loss: 0.000
0.428571428571
